In [1]:
import random 
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [3]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='UTF-8').lower()
text = text[300000:800000]
characters = sorted(set(text))
char_to_index = dict((c,i) for i,c in enumerate(characters))
index_to_char = dict((i,c) for i,c in enumerate(characters))


In [4]:
seq_length = 40
step_size = 3
sentences = []
next_characters = []
for i in range(0, len(text) - seq_length, step_size):
    sentences.append(text[i : i+seq_length])
    next_characters.append(text[i+seq_length])


In [8]:
x = np.zeros((len(sentences), seq_length, len(characters)), dtype=bool)
y = np.zeros((len(sentences), len(characters)), dtype=bool)
for i , sentence in enumerate(sentences):
    for t , character in enumerate(sentence):
        x[i, t, char_to_index[character]] = 1 #true = 1 
    y[i, char_to_index[next_characters[i]]] = 1


In [12]:
model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))
model.fit(x, y, batch_size=256, epochs=4)
          

Epoch 1/4
651/651 [==============================] - 174s 261ms/step - loss: 2.6959
Epoch 2/4
651/651 [==============================] - 130s 200ms/step - loss: 2.3045
Epoch 3/4
651/651 [==============================] - 112s 172ms/step - loss: 2.1876
Epoch 4/4
651/651 [==============================] - 96s 148ms/step - loss: 2.1069


In [13]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [21]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - seq_length - 1)
    generated = ''
    sentence = text[start_index: start_index + seq_length]
    generated += sentence
    for i in range(length):
        x = np.zeros((1, seq_length, len(characters)))
        for t, character in enumerate(sentence):
            x[0, t, char_to_index[character]] = 1
        predictions = model.predict(x, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_character = index_to_char[next_index]

        generated +=next_character
        sentence = sentence[1:] + next_character
        return generated


In [22]:
print(generate_text(300, 0.2))

se this house against this house,
it will


In [23]:
print(generate_text(300, 1.0))

k:
mine, full of sorrow and heart's discs
